In [87]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
from tqdm import tqdm

In [88]:
##===
# Select state we want to predict
# New York , California , Florida , Texas
STATE = 'Texas'
LR = 5e-5
WEIGHTDECAY = 0
TRAIN = True
seq_length = 7
EPOCHS = 5000
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
##===

### Import Data from CSV file

In [89]:
df = pd.read_csv('Cleaned_data.csv')

## Data frame Format

In [90]:
df.head()

,date,state,fips,cases,daily_cases,deaths,daily_deaths,total_vaccinations,total_distributed,daily_distributed,total_people_vaccinated,daily_people_vaccinated,total_people_fully_vaccinated,daily_people_fully_vaccinated,daily_vaccinations,mobility_composite
0,3/13/2021,Alabama,1,503305,594,10327,28,1288242,1918560.0,80160.0,841394.0,24976.0,492755.0,20430.0,"26,935",-1.013041
1,3/12/2021,Alabama,1,502711,448,10299,25,1249205,1838400.0,22170.0,816418.0,25554.0,472325.0,19579.0,"26,375",-1.013041
2,3/11/2021,Alabama,1,502263,865,10274,52,1209802,1816230.0,40570.0,790864.0,22362.0,452746.0,17728.0,"25,376",-1.013041
3,3/10/2021,Alabama,1,501398,782,10222,36,1173820,1775660.0,86610.0,768502.0,21234.0,435018.0,14935.0,"24,827",-1.013041
4,3/9/2021,Alabama,1,500616,524,10186,38,1140984,1689050.0,60700.0,747268.0,14301.0,420083.0,10146.0,"24,118",-1.013041


In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20618 entries, 0 to 20617
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   date                           20618 non-null  object 
 1   state                          20618 non-null  object 
 2   fips                           20618 non-null  int64  
 3   cases                          20618 non-null  int64  
 4   daily_cases                    20618 non-null  int64  
 5   deaths                         20618 non-null  int64  
 6   daily_deaths                   20618 non-null  int64  
 7   total_vaccinations             3801 non-null   object 
 8   total_distributed              3801 non-null   float64
 9   daily_distributed              3801 non-null   float64
 10  total_people_vaccinated        3801 non-null   float64
 11  daily_people_vaccinated        3801 non-null   float64
 12  total_people_fully_vaccinated  3801 non-null  

In [92]:
df = df.drop(labels=['cases', 'deaths', 'total_vaccinations', 'total_distributed', 'total_people_vaccinated', 'total_people_fully_vaccinated'], axis=1)
df.head()

,date,state,fips,daily_cases,daily_deaths,daily_distributed,daily_people_vaccinated,daily_people_fully_vaccinated,daily_vaccinations,mobility_composite
0,3/13/2021,Alabama,1,594,28,80160.0,24976.0,20430.0,"26,935",-1.013041
1,3/12/2021,Alabama,1,448,25,22170.0,25554.0,19579.0,"26,375",-1.013041
2,3/11/2021,Alabama,1,865,52,40570.0,22362.0,17728.0,"25,376",-1.013041
3,3/10/2021,Alabama,1,782,36,86610.0,21234.0,14935.0,"24,827",-1.013041
4,3/9/2021,Alabama,1,524,38,60700.0,14301.0,10146.0,"24,118",-1.013041


In [93]:
df['state'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Guam', 'Hawaii', 'Idaho', 'Illinois',
       'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine',
       'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Northern Mariana Islands',
       'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virgin Islands', 'Virginia',
       'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype=object)

In [94]:
# Set date to be datetime format and sort by date and fips
df['date'] = pd.to_datetime(df.date)
df = df.sort_values(by=['date', 'fips'], axis=0, ascending=True)
df

,date,state,fips,daily_cases,daily_deaths,daily_distributed,daily_people_vaccinated,daily_people_fully_vaccinated,daily_vaccinations,mobility_composite
19484,2020-01-21,Washington,53,1,0,NaN,NaN,NaN,NaN,NaN
19483,2020-01-22,Washington,53,0,0,NaN,NaN,NaN,NaN,NaN
19482,2020-01-23,Washington,53,0,0,NaN,NaN,NaN,NaN,NaN
5684,2020-01-24,Illinois,17,1,0,NaN,NaN,NaN,NaN,NaN
19481,2020-01-24,Washington,53,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
20250,2021-03-13,Wyoming,56,0,0,0.0,4086.0,2945.0,"4,085",0.000000
4168,2021-03-13,Guam,66,1,0,0.0,226.0,1440.0,"1,272",-5.361115
13475,2021-03-13,Northern Mariana Islands,69,1,0,0.0,230.0,163.0,246,NaN
15319,2021-03-13,Puerto Rico,72,235,0,19550.0,32455.0,20400.0,"19,615",0.000000


In [95]:
df_clean = df[df.state == STATE]
df_clean = df_clean.set_index('date')
df_clean

,state,fips,daily_cases,daily_deaths,daily_distributed,daily_people_vaccinated,daily_people_fully_vaccinated,daily_vaccinations,mobility_composite
date,,,,,,,,,
2020-02-12,Texas,48,1,0,NaN,NaN,NaN,NaN,-0.534251
2020-02-13,Texas,48,1,0,NaN,NaN,NaN,NaN,-0.062539
2020-02-14,Texas,48,0,0,NaN,NaN,NaN,NaN,0.373968
2020-02-15,Texas,48,0,0,NaN,NaN,NaN,NaN,0.754066
2020-02-16,Texas,48,0,0,NaN,NaN,NaN,NaN,1.069516
...,...,...,...,...,...,...,...,...,...
2021-03-09,Texas,48,5409,184,444420.0,168326.0,68666.0,"191,891",-7.686501
2021-03-10,Texas,48,5350,202,815040.0,165705.0,77828.0,"191,158",-7.686501
2021-03-11,Texas,48,6185,212,150650.0,153209.0,60099.0,"193,081",-7.686501


In [96]:
rmlabels=['state', 'fips', 'daily_deaths', 'daily_distributed','daily_people_vaccinated', 'daily_people_fully_vaccinated',
          'daily_vaccinations', 'mobility_composite']
df_clean = df_clean.drop(labels=rmlabels, axis=1)
df_clean = df_clean.iloc[df_clean.shape[0]-378:]
df_clean.shape

(378, 1)

In [97]:
df_clean.plot(title=f"Daily cases :{STATE}")

In [98]:
df.head()

,date,state,fips,daily_cases,daily_deaths,daily_distributed,daily_people_vaccinated,daily_people_fully_vaccinated,daily_vaccinations,mobility_composite
19484,2020-01-21,Washington,53,1,0,NaN,NaN,NaN,NaN,NaN
19483,2020-01-22,Washington,53,0,0,NaN,NaN,NaN,NaN,NaN
19482,2020-01-23,Washington,53,0,0,NaN,NaN,NaN,NaN,NaN
5684,2020-01-24,Illinois,17,1,0,NaN,NaN,NaN,NaN,NaN
19481,2020-01-24,Washington,53,0,0,NaN,NaN,NaN,NaN,NaN


In [99]:
df_state = df[df.state == STATE]

In [100]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

corrMatrix = df_state.corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()

### select the high correlation to be our factors

- daily death
- daily people vacinated 
- daily_distibuted  
- daily case 

In [101]:
df_state.columns

Index(['date', 'state', 'fips', 'daily_cases', 'daily_deaths',
       'daily_distributed', 'daily_people_vaccinated',
       'daily_people_fully_vaccinated', 'daily_vaccinations',
       'mobility_composite'],
      dtype='object')

In [102]:
df_state = df_state.set_index('date') 

In [103]:
df_state.head()

,state,fips,daily_cases,daily_deaths,daily_distributed,daily_people_vaccinated,daily_people_fully_vaccinated,daily_vaccinations,mobility_composite
date,,,,,,,,,
2020-02-12,Texas,48,1,0,NaN,NaN,NaN,NaN,-0.534251
2020-02-13,Texas,48,1,0,NaN,NaN,NaN,NaN,-0.062539
2020-02-14,Texas,48,0,0,NaN,NaN,NaN,NaN,0.373968
2020-02-15,Texas,48,0,0,NaN,NaN,NaN,NaN,0.754066
2020-02-16,Texas,48,0,0,NaN,NaN,NaN,NaN,1.069516


In [104]:
df_state = df_state.drop(['state', 'fips',
       'daily_people_vaccinated',
       'daily_people_fully_vaccinated',
       'mobility_composite'], axis=1)

In [105]:
df_state.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 396 entries, 2020-02-12 to 2021-03-13
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   daily_cases         396 non-null    int64  
 1   daily_deaths        396 non-null    int64  
 2   daily_distributed   72 non-null     float64
 3   daily_vaccinations  72 non-null     object 
dtypes: float64(1), int64(2), object(1)
memory usage: 15.5+ KB


In [106]:
df_state_clean = df_state.dropna()

In [107]:
df_state_clean = df_state_clean.replace(',','', regex=True)

In [108]:
df_state_clean.shape

(72, 4)

In [109]:
df_state_clean.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 72 entries, 2021-01-01 to 2021-03-13
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   daily_cases         72 non-null     int64  
 1   daily_deaths        72 non-null     int64  
 2   daily_distributed   72 non-null     float64
 3   daily_vaccinations  72 non-null     object 
dtypes: float64(1), int64(2), object(1)
memory usage: 2.8+ KB


In [120]:
df_state_clean.shape

(72, 4)

In [118]:
train_data = df_state_clean.iloc[:int(len(df_state_clean)*0.8),:]
test_data = df_state_clean.iloc[int(len(df_state_clean)*0.8):,:]
# train_target =  df_state_clean.iloc[:int(len(df_state_clean)*0.8),0]
# test_target = df_state_clean.iloc[int(len(df_state_clean)*0.8):,0]

In [121]:
assert train_data.shape[0] + test_data.shape[0] == df_state_clean.shape[0]

In [128]:
train_data.shape , len(train_data)-7 -1 

((57, 4), 49)

In [ ]:
# X start at 0 : length ,  
# y start at 0 + length : (i+1) * length

In [125]:
def create_sequences(data, seq_length):
    xs = []
    ys = []

    for i in range(len(data)-seq_length-1):
        x = data[i:(i+seq_length),:]
        y = data[i+seq_length,0]
        xs.append(x)
        ys.append(y)

    return np.array(xs), np.array(ys)

X_train, y_train = create_sequences(train_data, seq_length)
# X_test, y_test = create_sequences(test_data, seq_length)

# X_train = torch.from_numpy(X_train).float()
# y_train = torch.from_numpy(y_train).float()

# X_test = torch.from_numpy(X_test).float()
# y_test = torch.from_numpy(y_test).float()

KeyError: 7

In [42]:
# scaler = MinMaxScaler()
scaler = StandardScaler()

scaler = scaler.fit(train_data)

train_data = scaler.transform(train_data)

test_data = scaler.transform(test_data)

In [43]:
BATCH_SIZE = 7  #keeping it binary so it fits GPU
# #Train set loader
train_iterator = torch.utils.data.DataLoader(dataset=train_data, 
                                           batch_size=BATCH_SIZE, 
                                            shuffle=True)
#Test set loader
test_iterator = torch.utils.data.DataLoader(dataset=test_data, 
                                          batch_size=BATCH_SIZE, 
                                          shuffle=True)